# 🗄️ Tenant Databases

> Isolated per-tenant data storage with user profiles, permissions, and settings.

---

In [ ]:
#| default_exp db_tenant

In [ ]:
#| export

from fastsql import *
from fastcore.utils import *
from fh_saas.db_host import timestamp, gen_id
import urllib.parse
import os
import logging

# Module-level logger - configured by app via configure_logging()
logger = logging.getLogger(__name__)

## 🎯 Overview

Each tenant gets their own **isolated database** containing:

| Model | Purpose |
|-------|---------|
| 👤 `TenantUser` | Local user profiles linked to global identity |
| 🔐 `TenantPermission` | Fine-grained resource permissions |
| ⚙️ `TenantSettings` | Tenant-wide configuration |

---

## 🏗️ Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                      🏠 HOST DATABASE                           │
│  ┌─────────────┐                                                │
│  │ TenantCatalog │ ──► Maps tenant_id → database URL           │
│  └──────┬──────┘                                                │
└─────────┼───────────────────────────────────────────────────────┘
          │
          │  get_or_create_tenant_db(tenant_id)
          ▼
┌─────────────────────────────────────────────────────────────────┐
│                    🗄️ TENANT DATABASE                           │
│                    (Isolated per tenant)                        │
├─────────────────────────────────────────────────────────────────┤
│  👤 TenantUser      → Local profile (links to GlobalUser.id)   │
│  🔐 TenantPermission → Resource-level access control           │
│  ⚙️ TenantSettings   → Timezone, currency, feature flags       │
│  🪝 WebhookEvent     → Idempotent webhook processing           │
│  🔑 WebhookSecret    → HMAC secrets per webhook source         │
├─────────────────────────────────────────────────────────────────┤
│  📊 [Your App Tables] → Transactions, budgets, etc.            │
└─────────────────────────────────────────────────────────────────┘
```

**Key Principle:** Tenant data is completely isolated → No cross-tenant data leakage possible

## 🔌 Tenant Connection

> ⚠️ **Naming Convention**: Use **underscores** (not hyphens) in `tenant_id` values.
> 
> Database names are derived from `tenant_id` (e.g., `tenant_acme_001` → `t_tenant_acme_001_db`).
> PostgreSQL and SQLite identifiers work best with alphanumeric characters and underscores.
> 
> ✅ Good: `tenant_acme_001`, `tenant_finxplorer_prod`
> ❌ Avoid: `tenant-acme-001`, `tenant.finxplorer.prod`

In [ ]:
from nbdev.showdoc import show_doc

In [ ]:
#| export

def get_or_create_tenant_db(tenant_id: str, tenant_name: str = None):
    """Get or create a tenant database connection by tenant ID.
    
    ⚠️ IMPORTANT: Caller is responsible for closing the returned Database connection
    by calling `db.conn.close()` and `db.engine.dispose()` when done.
    """
    from sqlalchemy import text
    
    # Connect to host - read from environment
    DB_TYPE = os.getenv("DB_TYPE", "POSTGRESQL")
    DB_USER = os.getenv("DB_USER", "postgres")
    DB_PASS = os.getenv("DB_PASS", "")
    DB_HOST = os.getenv("DB_HOST", "localhost")
    DB_PORT = os.getenv("DB_PORT", "5432")
    DB_NAME = os.getenv("DB_NAME", "app_host")  # Host database name
    
    # Build host database connection
    if DB_TYPE == "POSTGRESQL":
        if not DB_PASS:
            raise ValueError("DB_PASS is required for PostgreSQL")
        encoded_pass = urllib.parse.quote_plus(DB_PASS)
        host_url = f"postgresql://{DB_USER}:{encoded_pass}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    else:
        host_url = f"sqlite:///{DB_NAME}.db"
    
    # Use try/finally to ensure host_db connection is always closed
    host_db = Database(host_url)
    try:
        # Check if tenant registered
        class TenantCatalog:
            id: str; name: str; db_url: str
            is_active: bool = True; plan_tier: str = "free"; created_at: str
        
        tenant_catalogs = host_db.create(TenantCatalog, name="core_tenants", pk='id')
        host_db.conn.rollback()
        all_tenants = tenant_catalogs()
        existing = [t for t in all_tenants if t.id == tenant_id]
        
        # Build tenant database connection
        # PostgreSQL databases cannot start with numbers, so prefix with 't_'
        if DB_TYPE == "POSTGRESQL":
            tenant_db_name = f"t_{tenant_id}_db"
            tenant_url = f"postgresql://{DB_USER}:{encoded_pass}@{DB_HOST}:{DB_PORT}/{tenant_db_name}"
        else:
            tenant_db_name = f"{tenant_id}_db"
            tenant_url = f"sqlite:///{tenant_db_name}.db"
        
        if not existing:
            print(f"⚡ Creating new tenant: {tenant_id}")
            
            # Create physical database (PostgreSQL only)
            if DB_TYPE == "POSTGRESQL":
                with host_db.engine.connect().execution_options(isolation_level="AUTOCOMMIT") as conn:
                    try:
                        conn.execute(text(f"CREATE DATABASE {tenant_db_name}"))
                        print(f"   ✅ Database created: {tenant_db_name}")
                    except Exception as e:
                        if "already exists" not in str(e):
                            raise
            
            # Register in host
            new_tenant = TenantCatalog(
                id=tenant_id,
                name=tenant_name or tenant_id,
                db_url=tenant_url,
                created_at=timestamp()
            )
            tenant_catalogs.insert(new_tenant)
            host_db.conn.commit()
            print(f"   ✅ Registered in host DB")
        else:
            print(f"ℹ️  Tenant exists: {existing[0].name}")
            tenant_url = existing[0].db_url
        
        return Database(tenant_url)
    finally:
        # Always close the internal host_db connection to prevent leaks
        try:
            host_db.conn.close()
            host_db.engine.dispose()
        except Exception:
            pass  # Ignore cleanup errors

`get_or_create_tenant_db()` handles the full lifecycle:

1. 🔍 **Check** if tenant exists in host database
2. 🗄️ **Create** physical database if new (PostgreSQL)
3. 📝 **Register** tenant in `TenantCatalog`
4. 🔌 **Return** connection to tenant database

| Parameter | Description |
|-----------|-------------|
| `tenant_id` | Unique tenant identifier (from `Membership`) |
| `tenant_name` | Optional display name (defaults to tenant_id) |

**Returns:** `Database` connection to the tenant's isolated database

In [ ]:
show_doc(get_or_create_tenant_db)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/db_tenant.py#L20){target="_blank" style="float:right; font-size:smaller"}

### get_or_create_tenant_db

>      get_or_create_tenant_db (tenant_id:str, tenant_name:str=None)

*Get or create a tenant database connection by tenant ID.*

---

## 📦 Core Tenant Models

These models provide the infrastructure every tenant needs. Your app-specific models (transactions, budgets, etc.) build on top of these.

In [ ]:
#| export

class TenantUser:
    """Local user profile linked to GlobalUser in host database."""
    id: str              # MUST match GlobalUser.id from host DB
    display_name: str    # e.g. "John Doe"
    local_role: str      # 'admin', 'editor', 'viewer'
    preferences: str = None  # JSON settings
    last_active: str = None
    created_at: str

class TenantPermission:
    """Fine-grained resource permission for a tenant user."""
    id: str
    user_id: str         # Links to TenantUser.id
    resource: str        # 'transactions', 'budgets', 'reports'
    action: str          # 'view', 'edit', 'delete'
    granted: bool = True
    created_at: str

class TenantSettings:
    """Tenant-wide configuration and feature flags."""
    id: str = "default"
    tenant_name: str
    timezone: str = "UTC"
    currency: str = "USD"
    feature_flags: str = None  # JSON
    updated_at: str

### 📝 Model Details

| Model | Table Name | Primary Key | Description |
|-------|------------|-------------|-------------|
| `TenantUser` | `core_tenant_users` | `id` | Links to `GlobalUser.id`, stores local role & preferences |
| `TenantPermission` | `core_permissions` | `id` | Resource + action permissions (RBAC) |
| `TenantSettings` | `core_settings` | `id` | Timezone, currency, feature flags |

---

## 🔧 Schema Initialization

In [ ]:
#| export

def init_tenant_core_schema(tenant_db: Database):
    """Create all core tenant tables and return table accessors."""
    tenant_users = tenant_db.create(TenantUser, name="core_tenant_users", pk='id')
    permissions = tenant_db.create(TenantPermission, name="core_permissions", pk='id')
    settings = tenant_db.create(TenantSettings, name="core_settings", pk='id')
    
    return {
        'tenant_users': tenant_users,
        'permissions': permissions,
        'settings': settings
    }

`init_tenant_core_schema()` creates all infrastructure tables in a tenant database:

```
tenant_db = get_or_create_tenant_db("tenant_abc")
tables = init_tenant_core_schema(tenant_db)

# Access tables via returned dict
tables['tenant_users'].insert(user)
tables['settings'].insert(settings)
```

**Returns:** Dictionary of table accessors for all core models

In [ ]:
show_doc(init_tenant_core_schema)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/db_tenant.py#L119){target="_blank" style="float:right; font-size:smaller"}

### init_tenant_core_schema

>      init_tenant_core_schema (tenant_db:fastsql.core.Database)

*Create all core tenant tables and return table accessors.*

---

## 🚀 Quick Start

```python
from fh_saas.db_tenant import get_or_create_tenant_db, init_tenant_core_schema, TenantUser
from fh_saas.db_host import gen_id, timestamp

# Get or create tenant database
tenant_db = get_or_create_tenant_db("tenant_abc123", "Acme Corp")

# Initialize core schema
tables = init_tenant_core_schema(tenant_db)

# Add a tenant user (linked to GlobalUser.id)
user = TenantUser(
    id="global_user_xyz",  # Must match GlobalUser.id
    display_name="John Doe",
    local_role="admin",
    created_at=timestamp()
)
tables['tenant_users'].insert(user)
tenant_db.conn.commit()
```

💡 **Tip:** The `id` field in `TenantUser` **must match** the `GlobalUser.id` from the host database to maintain identity linking.

---

## 🔐 Role-Based Access Control (RBAC)

Every tenant has a **three-tier role system** for controlling access:

### Role Hierarchy

| Role | Level | Description |
|------|-------|-------------|
| `admin` | 3 | Full access to all resources and settings |
| `editor` | 2 | Can view and modify data, but not settings |
| `viewer` | 1 | Read-only access to data |

### Role Assignment Rules

1. **Tenant owner** (from host `Membership.role='owner'`) → automatically gets `admin` role
2. **Other users** → assigned explicitly by admin via `TenantUser.local_role`
3. **No defaults** → admins must explicitly assign roles when inviting users

### TenantUser.local_role

The `local_role` field in `TenantUser` stores the user's role within the tenant:

```python
# Example: Admin adds a new user as editor
new_user = TenantUser(
    id=global_user_id,      # Must match GlobalUser.id
    display_name="Jane Doe",
    local_role="editor",    # Assigned by admin
    created_at=timestamp()
)
tables['tenant_users'].insert(new_user)
```

### Fine-Grained Permissions (Optional)

For advanced use cases, the `core_permissions` table allows resource-level control:

```python
# Example: Grant user edit access to transactions only
permission = TenantPermission(
    id=gen_id(),
    user_id=tenant_user.id,
    resource="transactions",
    action="edit",
    granted=True,
    created_at=timestamp()
)
tables['permissions'].insert(permission)
```

| Field | Description |
|-------|-------------|
| `resource` | What the permission applies to (e.g., "transactions", "budgets") |
| `action` | What action is allowed (e.g., "view", "edit", "delete") |
| `granted` | True = allowed, False = explicitly denied |

> 💡 **Tip:** Most apps only need the `local_role` field. Use `core_permissions` when you need per-resource control (e.g., "User X can view transactions but not budgets").

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()